<a href="https://colab.research.google.com/github/trefftzc/partition/blob/main/thrust_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update -qq

17 packages can be upgraded. Run 'apt list --upgradable' to see them.


In [ ]:
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb

--2023-11-30 02:53:55--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.nvidia.com/downloads/compute/cuda/8.0/prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb [following]
--2023-11-30 02:53:56--  https://developer.nvidia.com/downloads/compute/cuda/8.0/prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?F223fiIp0Cx-kxpbCQDHIMq-lj9fVDaFI9WK8iUIg-reJ-YSQN1gSXuhRvInhLIxDB8xU

In [ ]:
!dpkg -i cuda-repo-ubuntu1604–8–0-local-ga2_8.0.61–1_amd64-deb 2> /dev/null

In [ ]:
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub

gpg: can't open '/var/cuda-repo-8-0-local-ga2/7fa2af80.pub': No such file or directory


In [ ]:
!apt-get update -qq

In [ ]:
!apt --fix-broken install
!apt-get install cuda gcc-5 g++-5 -y -qq;

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 17 not upgraded.
E: Unable to locate package gcc-5
E: Package 'g++-5' has no installation candidate


In [ ]:
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package cuda-8.0
E: Couldn't find any package by glob 'cuda-8.0'


In [ ]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'

In [ ]:
!apt install gcc-5 g++-5 -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package g++-5 is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  gcc-10-test-results gcc-9-test-results gcc-12-test-results gcc-11-test-results

E: Unable to locate package gcc-5
E: Package 'g++-5' has no installation candidate


In [ ]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
%%file version.cu
#include <thrust/version.h>
#include <iostream>

int main(void)
{
  int major = THRUST_MAJOR_VERSION;
  int minor = THRUST_MINOR_VERSION;

  std::cout << "Thrust v" << major << "." << minor << std::endl;

  return 0;
}

Writing version.cu


In [ ]:
!nvcc version.cu -o version
!./version

Thrust v1.15


In [ ]:
%%file thrust_example.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/generate.h>
#include <thrust/sort.h>
#include <thrust/copy.h>
#include <algorithm>
#include <cstdlib>

int main(void)
{
  // generate 32M random numbers serially
  thrust::host_vector<int> h_vec(32 << 20);
  std::generate(h_vec.begin(), h_vec.end(), rand);

  // transfer data to the device
  thrust::device_vector<int> d_vec = h_vec;

  // sort data on the device
  thrust::sort(d_vec.begin(), d_vec.end());

  // transfer data back to host
  thrust::copy(d_vec.begin(), d_vec.end(), h_vec.begin());

  return 0;
}

Writing thrust_example.cu


In [ ]:
!nvcc thrust_example.cu -o thrust_example
!./thrust_example


In [ ]:
!nvprof ./thrust_example

==4180== NVPROF is profiling process 4180, command: ./thrust_example
==4180== Profiling application: ./thrust_example
==4180== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   36.73%  29.858ms         1  29.858ms  29.858ms  29.858ms  [CUDA memcpy DtoH]
                   36.59%  29.745ms         1  29.745ms  29.745ms  29.745ms  [CUDA memcpy HtoD]
                   13.93%  11.329ms         3  3.7762ms  3.7748ms  3.7771ms  void cub::DeviceRadixSortDownsweepKernel<cub::DeviceRadixSortPolicy<int, cub::NullType, int>::Policy800, bool=1, bool=0, int, cub::NullType, int>(cub::DeviceRadixSortPolicy<int, cub::NullType, int>::Policy800 const *, cub::DeviceRadixSortDownsweepKernel<cub::DeviceRadixSortPolicy<int, cub::NullType, int>::Policy800, bool=1, bool=0, int, cub::NullType, int>*, bool=1 const *, cub::DeviceRadixSortDownsweepKernel<cub::DeviceRadixSortPolicy<int, cub::NullType, int>::Policy800, bool=1, bool=0, int, cub::N

# Solving the partition problem with CUDA

The following source code solves the partition problem in CUDA

In [1]:
%%file cudaPartition.cu
/*
 * cudaPartition.cu
 * Solve the Partition problem using CUDA.
 * https://en.wikipedia.org/wiki/Partition_problem
 * This code works for multisets of up to 32 elements
 * The input is expected to be as follows:
 * The first line will contain n, the number of elements in the multiset
 * The remaining n lines will contain the n values, one per line
 */
#include <stdio.h>
#include <stdlib.h>
// The kernel
// This function is executed, in parallel, on the processors on the GPU card
//
__global__
void evaluatePartition(  int n, int *array,int *result) {
  unsigned int value = blockIdx.x*blockDim.x + threadIdx.x;
  int sum0s = 0;
  int sum1s = 0;
  unsigned int mask = 1;
  for(int i = 0;i < n;i++) {
    if ((mask & value) != 0) {
      sum1s = sum1s + array[i];
    }
    else {
      sum0s = sum0s + array[i];
    }
    mask = mask * 2;
  }
  if (sum0s == sum1s)
     result[value] = 1;
  else
     result[value] = 0;
  // printf("%d %d \n",value,result[value]);
}

void printResults(unsigned int value,int n,int *array)
{
  printf("Solution:\n");
  printf("First partition: ") ;
  unsigned int mask = 1;
  int sum = 0;
  for(int i = 0;i < n;i++) {
    if ((mask & value) != 0) {
      printf("%d ",array[i]);
      sum = sum + array[i];
    }
    mask = mask * 2;
  }
  printf(" sum: %d \n",sum);
  printf("Second partition: ") ;
  mask = 1;
  sum = 0;
  for(int i = 0;i < n;i++) {
    if ((mask & value) == 0) {
      printf("%d ",array[i]);
      sum = sum + array[i];
    }
    mask = mask * 2;
  }
  printf(" sum: %d \n",sum);
}


int main() {

  int n;
  int *array;

  scanf("%d",&n);

  printf("The value of n is %d\n",n);
  array = (int *) malloc (n * sizeof(int));
  for(int i = 0;i < n;i++) {
    scanf("%d",&array[i]);
  }
  printf("The read values are: \n");
  for(int i = 0;i < n;i++) {
    printf("%d ",array[i]);
  }
  printf("\n");

  unsigned int nPartitions = 1;
  for(int i = 0;i < n;i++) {
    nPartitions = nPartitions * 2;
  }
  // printf("The number of possible partitions is: %d\n",nPartitions);
  // Only half of all possible partitions need be examined
  // The second half is symmetrical to the first half
  nPartitions = nPartitions / 2;

  int solutionFound = 0;
  int solution = -1;
  // Allocate the variables in the device:
  // The array with the integer values in the device is called d_array
  int *d_array;
  cudaMalloc(&d_array, n*sizeof(int));

  // Copy the variables from the host to the device
  cudaMemcpy(d_array,array,n*sizeof(int), cudaMemcpyHostToDevice);

  // Allocate on the device an array to keep all the results
  int *d_results;
  cudaMalloc(&d_results,nPartitions*sizeof(int));
// Now invoke the kernel
  evaluatePartition<<<(nPartitions+31)/32,32>>>(  n, d_array,d_results) ;
  // The array on the host that will contain the results is called results
  int *results;
  results = (int *) calloc (nPartitions , sizeof(int));
// Copy the results from the GPU card to main memory on the host
  cudaMemcpy(results,d_results,nPartitions*sizeof(int),cudaMemcpyDeviceToHost);
  /*
  for(int i = 0;i < nPartitions;i++) {
	 printf("%d ",results[i]);
  }
  printf("\n");
 */
  for(int i = 0;i < nPartitions;i++) {
	  if (results[i] != 0) {
		  solutionFound = 1;
		  solution = i;
		  break;
	  }
  }

  if (solutionFound == 1) {
    printResults(solution, n, array);
  }
  else {
    printf("No solution was found.");
  }
  return 0;
}

Writing cudaPartition.cu


In [3]:
!nvcc cudaPartition.cu -o cudaPartition

A test file

In [4]:
%%writefile test20.txt
20
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 19

Writing test20.txt


Execute the program

In [5]:
!./cudaPartition < test20.txt

The value of n is 20
The read values are: 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 19 
Solution:
First partition: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1  sum: 19 
Second partition: 19  sum: 19 


Now a version with Thrust

Another, smaller, test file

In [46]:
%%writefile test4.txt
4
1 1 1 3

Writing test4.txt


In [98]:
%%writefile partition_thrust.cu
//
// The Thrust documentation is available at
// https://docs.nvidia.com/cuda/thrust/index.html
//

#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <iostream>

using namespace std;

struct evaluate_partition
{

    int  *array;
    int n;

    __host__
    __device__

    evaluate_partition(int * _array,int _n) : array(_array),n(_n)  {}

     __host__
     __device__

    int operator()(const int value)  {

    int sum0s = 0;
    int sum1s = 0;
    unsigned int mask = 1;
    // int n = array.size();
    for(int i = 0;i < n;i++) {
      if ((mask & value) != 0) {
        sum1s = sum1s + array[i];
      }
      else {
        sum0s = sum0s + array[i];
      }
      mask = mask * 2;
    }

    if (sum0s == sum1s)
      return 1;
    else
      return 0;


    }
};


int main() {
    int n;
    cin >> n;
    cout << "The size of the problem is: " << n << endl;

    thrust::host_vector <int> values_host(n);
    for(int i = 0;i < n;i++) {
      cin >> values_host[i];
    }
    cout << "The problem is: " << endl;
    for(int i = 0;i < n;i++) {
      cout << values_host[i] << " ";
    }
    cout << endl;
    thrust::device_vector < int > values_device(n);

    values_device = values_host;

    int size_power_set = 1;
    for(int i = 0;i < n;i++) {
      size_power_set = size_power_set * 2;
    }
    cout << "The size of the power set is " << size_power_set << endl;

    thrust::device_vector < int > result_device(size_power_set);
    thrust::host_vector < int > input_host(size_power_set);
    thrust::host_vector < int > result_host(size_power_set);
    /*
    thrust::counting_iterator<int> first(0);
    thrust::counting_iterator<int> last = first + size_power_set;
    */
    thrust::sequence(input_host.begin(),input_host.end());
    cout << "input_host: " << endl;
    for(int i = 0;i < size_power_set;i++) {
      cout << input_host[i] << " ";
    }
    cout << endl;

    thrust::device_vector < int > input_device(size_power_set);

    input_device = input_host;

    evaluate_partition eval_part(thrust::raw_pointer_cast(values_device.data()),n);
    thrust::transform(input_device.begin(),input_device.end(),
                          result_device.begin(),eval_part);

    int result = thrust::reduce(result_device.begin(),result_device.end(),(int)0, thrust::plus<int>());

    result_host = result_device;
    cout << "Result host: " << endl;
    for(int i = 0;i < size_power_set;i++) {
      cout << result_host[i] << " ";
    }
    cout << endl;

    if (result == 0) {
      cout << "This instance does not have a solution." << endl;
    }
    else {
      cout << "This instance has a solution." << endl;
    }

}

Overwriting partition_thrust.cu


In [99]:
!nvcc partition_thrust.cu -o partition_thrust

In [100]:
!./partition_thrust < test4.txt

The size of the problem is: 4
The problem is: 
1 1 1 3 
The size of the power set is 16
input_host: 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 
Result host: 
0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 
This instance has a solution.
